# Sentiment Analysis con KNN
## Clasificador en C++ 💪💪
Vamos a probar a nuestro bichito


Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [1]:
!cd .. && git submodule init
!cd .. && git submodule update
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

mkdir: cannot create directory ‘build’: File exists
-- The C compiler identification is GNU 7.4.0
-- The CXX compiler identification is GNU 7.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp: /usr/bin/python (found version "2.7.15") 
-- Found PythonLibs: /usr/lib/x86_64-linux-gnu/libpython2.7.so
-- pybind11 v2.3.dev0
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- LTO enabled
CMAKE_INSTALL_PREFIX=/home2/clinux01/Downloads/git
-- Configuring done
-- Generating done
-- Build files have 

In [4]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python3 --version
import sentiment


/home2/clinux01/Downloads/git/notebooks
Python 3.6.8


In [5]:
%load_ext autoreload
%autoreload 2
import pandas as pd

!cd ../data && tar -xvf *.tgz

df = pd.read_csv("../data/imdb_small.csv", index_col=0)

print("Cantidad de documentos: {}".format(df.shape[0]))

tar: *.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
Cantidad de documentos: 12500


In [6]:
df.describe()

,type,review,label,file
count,12500,12500,12500,12500
unique,2,12469,2,12085
top,test,This movie has made me upset! When I think of ...,neg,6905_1.txt
freq,6275,2,6322,2


In [7]:
text_train = df[df.type == 'train']["review"]
label_train = df[df.type == 'train']["label"]
text_train = text_train[0:200]
label_train = label_train[0:200]

text_test = df[df.type == 'test']["review"]
label_test = df[df.type == 'test']["label"]
text_test = text_test[0:200]
label_test = label_test[0:200]

print("Cantidad de instancias de entrenamiento = {}".format(len(text_train)))
print("Cantidad de instancias de test = {}".format(len(text_test)))


Cantidad de instancias de entrenamiento = 10
Cantidad de instancias de test = 200


In [8]:
print("Class balance : {} pos {} neg".format(
    (label_train == 'pos').sum(),
    (label_train == 'neg').sum() ))


Class balance : 5 pos 5 neg


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.90, min_df=0.01, max_features=5000)

vectorizer.fit(text_train)

X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values

In [ ]:
#ESTO SUPUESTAMENTE ENTRENA PCA

import sentiment
from sentiment import PCA, KNNClassifier
alpha = 6
pca = PCA(alpha)
pca.fit(X_train.toarray())
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [10]:
%%time

import sentiment
from sklearn.metrics import accuracy_score

for k in range(1,100,3):
    
    clf = sentiment.KNNClassifier(k)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Accuracy para {}: {}".format(k,acc))


Accuracy para 1: 0.53
Accuracy para 4: 0.53
Accuracy para 7: 0.545
Accuracy para 10: 0.54
Accuracy para 13: 0.585
Accuracy para 16: 0.58
Accuracy para 19: 0.56
Accuracy para 22: 0.6
Accuracy para 25: 0.59
Accuracy para 28: 0.525
Accuracy para 31: 0.58
Accuracy para 34: 0.54
Accuracy para 37: 0.61
Accuracy para 40: 0.515
Accuracy para 43: 0.55
Accuracy para 46: 0.55
Accuracy para 49: 0.565
Accuracy para 52: 0.58
Accuracy para 55: 0.545
Accuracy para 58: 0.565
Accuracy para 61: 0.595
Accuracy para 64: 0.52
Accuracy para 67: 0.595
Accuracy para 70: 0.595
Accuracy para 73: 0.51
Accuracy para 76: 0.5
Accuracy para 79: 0.55
Accuracy para 82: 0.535
Accuracy para 85: 0.575
Accuracy para 88: 0.58
Accuracy para 91: 0.585
Accuracy para 94: 0.485
Accuracy para 97: 0.505


In [24]:
%%time
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test)
print(y_pred)


acc = accuracy_score(y_test, y_pred)
print("Accuracy: {}".format(acc))

[1. 0. 0. ... 0. 1. 1.]
Accuracy: 0.5469322709163347
CPU times: user 49min 29s, sys: 18.6 s, total: 49min 48s
Wall time: 49min 23s
